In [1]:
import pandas as pd
from scipy.sparse import dok_matrix, save_npz
from tqdm import tqdm
tqdm.pandas()  # Enable tqdm for pandas
import json
DGIDB_PATH = "./Data/DGIDB/interactions.tsv"
DGIDB = pd.read_csv(DGIDB_PATH, sep="\t")
HUMANNET_PATH = "./Data/HumanNet/HumanNet-GSP.tsv"
HUMANNET = pd.read_csv(HUMANNET_PATH, sep="\t")
NCBI_PATH = "./Data/ncbi/gene2refseq.gz"
NCBI_INFO = pd.read_csv(NCBI_PATH, sep='\t', compression='gzip')

c:\Users\richa\Desktop\CodingWorkspaces\DGI-Hypergraph\datatestingvenv\lib\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (2,4,6,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:

# Filter for Homo sapiens genes (tax_id = 9606)
human_gene2refseq = NCBI_INFO[NCBI_INFO['#tax_id'] == 9606]

# Create a mapping from gene claim names (Symbol) to NCBI gene IDs (GeneID)
gene_claim_to_id = pd.Series(human_gene2refseq.GeneID.values, index=human_gene2refseq.Symbol).to_dict()
gene_claim_name = "HTR2C"
ncbi_gene_id = gene_claim_to_id.get(gene_claim_name, "Gene name not found")
print(f"NCBI Gene ID for {gene_claim_name}: {ncbi_gene_id}")

NCBI Gene ID for HTR2C: 3358


In [3]:
def get_ncbi_gene_id(gene_claim_name):
    return gene_claim_to_id.get(gene_claim_name, None)
# Add a new column to dgidb with NCBI gene IDs


In [4]:
DGIDB['ncbi_gene_id'] = DGIDB['gene_name'].apply(get_ncbi_gene_id)
print(DGIDB.head())

  gene_claim_name gene_concept_id gene_name interaction_source_db_name  \
0          CYP2D6       hgnc:2625    CYP2D6                        DTC   
1           PPARG       hgnc:9236     PPARG                        DTC   
2           ATAD5      hgnc:25752     ATAD5                        DTC   
3            RGS4      hgnc:10000      RGS4                        DTC   
4           MAPK1       hgnc:6871     MAPK1                        DTC   

  interaction_source_db_version interaction_type  interaction_score  \
0                        9/2/20              NaN           0.017709   
1                        9/2/20              NaN           0.840123   
2                        9/2/20              NaN           0.177992   
3                        9/2/20              NaN           0.034319   
4                        9/2/20              NaN           0.050007   

           drug_claim_name       drug_concept_id                drug_name  \
0               RACLOPRIDE          ncit:C152139   

In [5]:
DGIDB['ncbi_gene_id'] = DGIDB['ncbi_gene_id'].apply(lambda x: str(int(x)) if pd.notnull(x) else None)
print(DGIDB.head())

  gene_claim_name gene_concept_id gene_name interaction_source_db_name  \
0          CYP2D6       hgnc:2625    CYP2D6                        DTC   
1           PPARG       hgnc:9236     PPARG                        DTC   
2           ATAD5      hgnc:25752     ATAD5                        DTC   
3            RGS4      hgnc:10000      RGS4                        DTC   
4           MAPK1       hgnc:6871     MAPK1                        DTC   

  interaction_source_db_version interaction_type  interaction_score  \
0                        9/2/20              NaN           0.017709   
1                        9/2/20              NaN           0.840123   
2                        9/2/20              NaN           0.177992   
3                        9/2/20              NaN           0.034319   
4                        9/2/20              NaN           0.050007   

           drug_claim_name       drug_concept_id                drug_name  \
0               RACLOPRIDE          ncit:C152139   

In [9]:
# Count rows where 'ncbi_gene_id' is None
none_count = DGIDB['ncbi_gene_id'].isna().sum()

# Print the count
print(f"Number of rows with None in 'ncbi_gene_id': {none_count}")
none_rows = DGIDB[DGIDB['ncbi_gene_id'].isna()]
none_rows.to_csv("none_rows.csv")

Number of rows with None in 'ncbi_gene_id': 9755


In [10]:
DGIDB.to_csv("./dgidb_ncbi_v1.csv", index=False)

In [7]:
ncbi_count = DGIDB['gene_claim_name'].str.contains('ncbi', case=False, na=False).sum()

print("Number of genes where gene_claim_name has 'ncbi':", ncbi_count)
ncbi_none_count = DGIDB[
    DGIDB['gene_claim_name'].str.contains('NCBIGENE:', case=False, na=False) &
    DGIDB['ncbi_gene_id'].isna()
].shape[0]

print("Number of genes where gene_claim_name has 'ncbi' and ncbi_gene_id is None:", ncbi_none_count)

Number of genes where gene_claim_name has 'ncbi': 16070
Number of genes where gene_claim_name has 'ncbi' and ncbi_gene_id is None: 1422


In [ ]:
# DGIDB.loc[
#     DGIDB['gene_claim_name'].str.contains('NCBIGENE:', case=False, na=False), 
#     'ncbi_gene_id'
# ] = DGIDB['gene_claim_name'].str.extract(r'NCBIGENE:(\d+)', expand=False)
# DGIDB['ncbi_gene_id'] = DGIDB['ncbi_gene_id'].apply(lambda x: str(int(x)) if pd.notnull(x) else None)

In [14]:
DGIDB_cleaned = DGIDB.dropna(subset=['ncbi_gene_id'])
print(DGIDB_cleaned.head())

  gene_claim_name gene_concept_id gene_name interaction_source_db_name  \
0          CYP2D6       hgnc:2625    CYP2D6                        DTC   
1           PPARG       hgnc:9236     PPARG                        DTC   
2           ATAD5      hgnc:25752     ATAD5                        DTC   
3            RGS4      hgnc:10000      RGS4                        DTC   
4           MAPK1       hgnc:6871     MAPK1                        DTC   

  interaction_source_db_version interaction_type  interaction_score  \
0                        9/2/20              NaN           0.017709   
1                        9/2/20              NaN           0.840123   
2                        9/2/20              NaN           0.177992   
3                        9/2/20              NaN           0.034319   
4                        9/2/20              NaN           0.050007   

           drug_claim_name       drug_concept_id                drug_name  \
0               RACLOPRIDE          ncit:C152139   

In [13]:
print(DGIDB_cleaned.shape)

(88484, 14)


In [15]:
DGIDB_cleaned.to_csv("dgidb_ncbi_v2.csv", index=False)
DGIDB = DGIDB_cleaned

In [14]:
# Convert ncbi_gene_id in DGIDB to a set for efficient lookup (drop NaN values)
dgidb_ncbi_ids = set(DGIDB['ncbi_gene_id'].dropna().astype(str))

# Combine HumanNet's gene1 and gene2 into a single set
humannet_genes = set(HUMANNET['Gene1'].astype(str)) | set(HUMANNET['Gene2'].astype(str))
humannet_gene1 = set(HUMANNET['Gene1'].astype(str))
# Find ncbi_gene_ids in DGIDB that are not in HumanNet
unique_in_dgidb1 = dgidb_ncbi_ids - humannet_gene1
unique_in_dgidb = dgidb_ncbi_ids - humannet_genes
# Print the count of such ncbi_gene_ids
print("Number of ncbi_gene_id in DGIDB but not in HumanNet[gene1]:", len(unique_in_dgidb1))
print("Number of ncbi_gene_id in DGIDB but not in HumanNet[gene1] or HumanNet[gene2]:", len(unique_in_dgidb))
print("Total number of interactions in DGIDB:", len(DGIDB))
print("Total number of unique genes in DGIDB:",len(dgidb_ncbi_ids))
rows_not_in_humannet = DGIDB[DGIDB['ncbi_gene_id'].notna() & ~DGIDB['ncbi_gene_id'].astype(str).isin(humannet_genes)]
print("Number of rows in DGIDB where ncbi_gene_id is not in HumanNet[gene1] or HumanNet[gene2]:", len(rows_not_in_humannet))


Number of ncbi_gene_id in DGIDB but not in HumanNet[gene1]: 2161
Number of ncbi_gene_id in DGIDB but not in HumanNet[gene1] or HumanNet[gene2]: 2032
Total number of interactions in DGIDB: 98239
Total number of unique genes in DGIDB: 4835
Number of rows in DGIDB where ncbi_gene_id is not in HumanNet[gene1] or HumanNet[gene2]: 17100


In [3]:
print(len(HUMANNET))

260962


In [28]:
v2 = pd.read_csv("./Data/DGIDB/converted/human/dgidb_ncbi_v2.csv")

In [29]:
none_rows = v2[v2['ncbi_gene_id'].isna()]
print(none_rows.head())

                                  gene_claim_name gene_concept_id gene_name  \
5                  ANGIOTENSIN II RECEPTOR TYPE-1        hgnc:336     AGTR1   
6   VASCULAR ENDOTHELIAL GROWTH FACTOR RECEPTOR 2       hgnc:6307       KDR   
7                    POLY [ADP-RIBOSE] POLYMERASE        hgnc:270     PARP1   
19                                 UNIPROT:P08588        hgnc:285     ADRB1   
20                                 UNIPROT:Q9Y5Y9      hgnc:10582    SCN10A   

   interaction_source_db_name interaction_source_db_version interaction_type  \
5                         TTD                    2020.06.01              NaN   
6                         TTD                    2020.06.01              NaN   
7                         TTD                    2020.06.01              NaN   
19                       TEND                      1-Aug-11              NaN   
20                       TEND                      1-Aug-11              NaN   

    interaction_score drug_claim_name   drug

In [30]:
# Remove rows with None values in 'ncbi_gene_id' column
v2 = v2.dropna(subset=['ncbi_gene_id'])

# Verify the number of rows removed
print(f"Number of rows removed: {len(none_rows)}")
print(f"Number of rows remaining: {len(v2)}")
v2['ncbi_gene_id'] = v2['ncbi_gene_id'].apply(lambda x: str(int(x)) if pd.notnull(x) else None)

Number of rows removed: 28195
Number of rows remaining: 70044


In [31]:
v2.to_csv("./Data/DGIDB/converted/human/dgidb_ncbi_v3.csv", index=False)

In [ ]:
import pandas as pd
DGIDB = pd.read_csv("./Data/DGIDB/converted/human/dgidb_ncbi_v3.csv")

c:\Users\richa\Desktop\CodingWorkspaces\DGI-Hypergraph\datatestingvenv\lib\site-packages\IPython\core\interactiveshell.py:3258: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# Convert ncbi_gene_id in DGIDB to a set for efficient lookup (drop NaN values)
dgidb_ncbi_ids = set(DGIDB['ncbi_gene_id'].dropna().astype(str))

# Combine HumanNet's gene1 and gene2 into a single set
humannet_genes = set(HUMANNET['Gene1'].astype(str)) | set(HUMANNET['Gene2'].astype(str))
humannet_gene1 = set(HUMANNET['Gene1'].astype(str))
# Find ncbi_gene_ids in DGIDB that are not in HumanNet
unique_in_dgidb1 = dgidb_ncbi_ids - humannet_gene1
unique_in_dgidb = dgidb_ncbi_ids - humannet_genes
# Print the count of such ncbi_gene_ids
print("Number of ncbi_gene_id in DGIDB but not in HumanNet[gene1]:", len(unique_in_dgidb1))
print("Number of ncbi_gene_id in DGIDB but not in HumanNet[gene1] or HumanNet[gene2]:", len(unique_in_dgidb))
print("Total number of interactions in DGIDB:", len(DGIDB))
print("Total number of unique genes in DGIDB:",len(dgidb_ncbi_ids))
rows_not_in_humannet = DGIDB[DGIDB['ncbi_gene_id'].notna() & ~DGIDB['ncbi_gene_id'].astype(str).isin(humannet_genes)]
print("Number of rows in DGIDB where ncbi_gene_id is not in HumanNet[gene1] or HumanNet[gene2]:", len(rows_not_in_humannet))


NameError: name 'HUMANNET' is not defined

In [21]:
import pandas as pd

# Convert ncbi_gene_id in DGIDB to a set for efficient lookup (drop NaN values)
dgidb_ncbi_ids = set(DGIDB['ncbi_gene_id'].dropna().astype(str))

# Combine HumanNet's gene1 and gene2 into a single set
humannet_genes = set(HUMANNET['Gene1'].astype(str)) | set(HUMANNET['Gene2'].astype(str))

# Find ncbi_gene_ids in DGIDB that are not in HumanNet
unique_in_dgidb = dgidb_ncbi_ids - humannet_genes

# Print the count of such ncbi_gene_ids
print("Number of ncbi_gene_id in DGIDB but not in HumanNet[gene1] or HumanNet[gene2]:", len(unique_in_dgidb))

# Optional: If you need the specific rows in DGIDB where ncbi_gene_id is not in HumanNet[gene1] or Human

Number of ncbi_gene_id in DGIDB but not in HumanNet[gene1] or HumanNet[gene2]: 2032


In [14]:
import pandas as pd


# Create a sorted tuple for each row
HUMANNET['GenePair'] = HUMANNET.apply(lambda x: tuple(sorted([x['Gene1'], x['Gene2']])), axis=1)

pair_counts = HUMANNET['GenePair'].value_counts()

# Identify duplicates (pairs occurring more than once)
duplicates = pair_counts[pair_counts > 1]

# Print the duplicate pairs
duplicate_pairs = duplicates.index.tolist()

print('Duplicate pairs:')
for pair in duplicate_pairs:
    print(pair)

# Number of duplicate pairs
num_duplicates = len(duplicates)

print(f'Number of duplicate pairs: {num_duplicates}')
# Count distinct pairs
unique_pairs = HUMANNET['GenePair'].nunique()

print(f'Number of unique pairs: {unique_pairs}')

Duplicate pairs:
(5293, 5297)
(5288, 5297)
(7324, 11065)
(7186, 7319)
(2182, 10965)
(4831, 51727)
(3093, 11065)
(3295, 116285)
(2235, 5498)
(2027, 7167)
(5947, 9249)
(6713, 7108)
(2530, 4247)
(4952, 56623)
(10555, 57678)
(7186, 63893)
(10682, 50814)
(6713, 50814)
(6500, 8454)
(7328, 27338)
(7328, 7332)
(2947, 2949)
(60481, 142827)
(2182, 341392)
(6470, 6472)
(2683, 6484)
(2592, 130589)
(51703, 122970)
(6483, 6489)
(5728, 8897)
(222, 10327)
(8382, 50484)
(7319, 27338)
(26229, 64131)
(2525, 2528)
(5294, 5295)
(2181, 60481)
(3093, 27338)
(9468, 56994)
(9562, 253430)
(6342, 11001)
(2524, 9331)
(8877, 56624)
(6483, 6487)
(51109, 83758)
(7324, 29089)
(4832, 50484)
(26229, 64132)
(5207, 57103)
(26289, 122481)
(2941, 2949)
(2678, 2950)
(11065, 51619)
(997, 7326)
(5297, 8396)
(81616, 123876)
(150763, 154141)
(54995, 122970)
(3930, 6713)
(2524, 10317)
(5305, 8396)
(5730, 5740)
(50515, 79586)
(217, 218)
(7321, 7326)
(23305, 123876)
(290, 221357)
(56548, 166012)
(7186, 7335)
(3033, 3295)
(7323, 73

In [ ]:
# UNDIRECTED
# # Create PPI subnetwork

# import numpy as np
# # Get the unique list of genes
# genes = pd.unique(HUMANNET[['Gene1', 'Gene2']].values.ravel())

# # Create a mapping of genes to indices for the adjacency matrix
# gene_to_index = {gene: idx for idx, gene in enumerate(genes)}

# # Initialize an adjacency matrix of zeros
# adj_matrix = np.zeros((len(genes), len(genes)), dtype=int)

# # Fill the adjacency matrix based on HumanNet edges
# for _, row in HUMANNET.iterrows():
#     i, j = gene_to_index[row["Gene1"]], gene_to_index[row["Gene2"]]
#     adj_matrix[i, j] = 1
#     adj_matrix[j, i] = 1  # Ensure symmetry

# # Create a DataFrame for the adjacency matrix for easy visualization if needed
# adj_matrix_df = pd.DataFrame(adj_matrix, index=genes, columns=genes)

# # Define a function to get the degree of a specific gene
# def get_gene_degree(gene):
#     if gene not in gene_to_index:
#         raise ValueError(f"Gene {gene} not found in the dataset.")
#     index = gene_to_index[gene]
#     return adj_matrix[index].sum()

# # Example usage
# gene_example = 54936
# degree = get_gene_degree(gene_example)
# print(f"The degree of gene {gene_example} is: {degree}")

The degree of gene 54936 is: 48


In [2]:
import pandas as pd
DGIDB = pd.read_csv("./Data/DGIDB/converted/human/dgidb_ncbi_v2.csv")
HUMANNET_PATH = "./Data/HumanNet/HumanNet-GSP.tsv"
HUMANNET = pd.read_csv(HUMANNET_PATH, sep="\t")



In [3]:
import numpy as np
import pandas as pd

# Get the unique list of genes
genes = pd.unique(HUMANNET[['Gene1', 'Gene2']].values.ravel())

# Create a mapping of genes to indices for the adjacency matrix
gene_to_index = {gene: idx for idx, gene in enumerate(genes)}

# Initialize a directed adjacency matrix of zeros
adj_matrix = np.zeros((len(genes), len(genes)), dtype=int)

# Fill the adjacency matrix based on HumanNet edges (directed)
for _, row in HUMANNET.iterrows():
    i, j = gene_to_index[row["Gene1"]], gene_to_index[row["Gene2"]]
    adj_matrix[i, j] = 1  # Directed edge from Gene1 → Gene2

# Create a DataFrame for easy visualization if needed
adj_matrix_df = pd.DataFrame(adj_matrix, index=genes, columns=genes)

# Define a function to get the out-degree of a specific gene
def get_gene_out_degree(gene):
    if gene not in gene_to_index:
        raise ValueError(f"Gene {gene} not found in the dataset.")
    index = gene_to_index[gene]
    return adj_matrix[index].sum()  # Sum across row → Out-degree

# Define a function to get the in-degree of a specific gene
def get_gene_in_degree(gene):
    if gene not in gene_to_index:
        raise ValueError(f"Gene {gene} not found in the dataset.")
    index = gene_to_index[gene]
    return adj_matrix[:, index].sum()  # Sum across column → In-degree

# Example usage
gene_example = 54936
out_degree = get_gene_out_degree(gene_example)
in_degree = get_gene_in_degree(gene_example)

print(f"The out-degree of gene {gene_example} is: {out_degree}")
print(f"The in-degree of gene {gene_example} is: {in_degree}")


The out-degree of gene 54936 is: 23
The in-degree of gene 54936 is: 25


In [4]:
from scipy.sparse import dok_matrix, save_npz
# Calculate gene degrees in HumanNet
genes_in_humannet = pd.unique(HUMANNET[['Gene1', 'Gene2']].values.ravel())
gene_to_degree = {gene: 0 for gene in genes_in_humannet}

for _, row in HUMANNET.iterrows():
    gene_to_degree[row["Gene1"]] += 1
    gene_to_degree[row["Gene2"]] += 1

# Add degrees to DGIDB with fallback to 0.01 for missing genes
DGIDB['ncbi_gene_id'] = DGIDB['ncbi_gene_id'].astype(str)
gene_to_degree = {str(gene): degree for gene, degree in gene_to_degree.items()}
DGIDB['degree'] = DGIDB['ncbi_gene_id'].map(gene_to_degree).fillna(0.01)

# Create mappings for vertices and hyperedges
genes = DGIDB['ncbi_gene_id'].unique()
drugs = DGIDB['drug_name'].unique()
gene_to_index = {gene: i for i, gene in enumerate(genes)}
drug_to_index = {drug: i for i, drug in enumerate(drugs)}

# Initialize a sparse incidence matrix
incidence_matrix = dok_matrix((len(genes), len(drugs)), dtype=float)

# Populate the incidence matrix with degree as weight
for _, row in DGIDB.iterrows():
    gene_idx = gene_to_index[row['ncbi_gene_id']]
    drug_idx = drug_to_index[row['drug_name']]
    degree = row['degree']
    incidence_matrix[gene_idx, drug_idx] = degree

# Convert the DOK matrix to CSR format
csr_matrix = incidence_matrix.tocsr()

# Save the matrix as .npz file
save_npz("hypergraph_incidence_matrix_weighted.npz", csr_matrix)

# Print confirmation
print("Weighted incidence matrix saved as 'hypergraph_incidence_matrix_weighted.npz'.")

Weighted incidence matrix saved as 'hypergraph_incidence_matrix_weighted.npz'.


In [5]:
import json

# Define file paths
gene_to_index_path = "gene_to_index.json"
drug_to_index_path = "drug_to_index.json"

# Save gene_to_index mapping
with open(gene_to_index_path, 'w') as gene_file:
    json.dump(gene_to_index, gene_file, indent=4)

# Save drug_to_index mapping
with open(drug_to_index_path, 'w') as drug_file:
    json.dump(drug_to_index, drug_file, indent=4)

print(f"Mappings saved to {gene_to_index_path} and {drug_to_index_path}.")


Mappings saved to gene_to_index.json and drug_to_index.json.


In [6]:
from scipy.sparse import dok_matrix, save_npz
import pandas as pd

# Create mappings for vertices and hyperedges
genes = DGIDB['ncbi_gene_id'].unique()
drugs = DGIDB['drug_name'].unique()
gene_to_index = {gene: i for i, gene in enumerate(genes)}
drug_to_index = {drug: i for i, drug in enumerate(drugs)}

# Initialize a sparse binary incidence matrix
binary_incidence_matrix = dok_matrix((len(genes), len(drugs)), dtype=int)

# Populate the binary incidence matrix with 1 where there is a relationship
for _, row in DGIDB.iterrows():
    gene_idx = gene_to_index[row['ncbi_gene_id']]
    drug_idx = drug_to_index[row['drug_name']]
    binary_incidence_matrix[gene_idx, drug_idx] = 1

# Convert the DOK matrix to CSR format
binary_csr_matrix = binary_incidence_matrix.tocsr()

# Save the binary matrix as .npz file
save_npz("hypergraph_incidence_matrix_binary.npz", binary_csr_matrix)

# Print confirmation
print("Binary incidence matrix saved as 'hypergraph_incidence_matrix_binary.npz'.")


Binary incidence matrix saved as 'hypergraph_incidence_matrix_binary.npz'.


In [7]:
print(binary_incidence_matrix)


  (0, 0)	1
  (1, 1)	1
  (2, 2)	1
  (3, 3)	1
  (4, 4)	1
  (5, 5)	1
  (6, 6)	1
  (7, 7)	1
  (8, 8)	1
  (9, 9)	1
  (10, 10)	1
  (11, 11)	1
  (12, 12)	1
  (13, 13)	1
  (14, 14)	1
  (15, 15)	1
  (16, 16)	1
  (17, 17)	1
  (18, 18)	1
  (19, 19)	1
  (20, 20)	1
  (5, 21)	1
  (21, 22)	1
  (22, 23)	1
  (23, 24)	1
  :	:
  (1501, 9075)	1
  (86, 14658)	1
  (140, 5772)	1
  (140, 4193)	1
  (53, 6616)	1
  (53, 3484)	1
  (4483, 316)	1
  (323, 246)	1
  (22, 2076)	1
  (635, 17906)	1
  (94, 9299)	1
  (94, 9969)	1
  (313, 3190)	1
  (813, 1005)	1
  (250, 36)	1
  (250, 3873)	1
  (216, 74)	1
  (183, 3600)	1
  (183, 2109)	1
  (1467, 17224)	1
  (364, 7502)	1
  (364, 13885)	1
  (364, 9299)	1
  (364, 1289)	1
  (106, 17907)	1


In [8]:
print(incidence_matrix)

  (0, 0)	52.0
  (1, 1)	398.0
  (2, 2)	16.0
  (3, 3)	0.01
  (4, 4)	202.0
  (5, 5)	192.0
  (6, 6)	362.0
  (7, 7)	257.0
  (8, 8)	0.01
  (9, 9)	0.01
  (10, 10)	54.0
  (11, 11)	26.0
  (12, 12)	311.0
  (13, 13)	41.0
  (14, 14)	67.0
  (15, 15)	0.01
  (16, 16)	158.0
  (17, 17)	59.0
  (18, 18)	0.01
  (19, 19)	79.0
  (20, 20)	46.0
  (5, 21)	192.0
  (21, 22)	324.0
  (22, 23)	138.0
  (23, 24)	62.0
  :	:
  (1501, 9075)	23.0
  (86, 14658)	0.01
  (140, 5772)	138.0
  (140, 4193)	138.0
  (53, 6616)	152.0
  (53, 3484)	152.0
  (4483, 316)	0.01
  (323, 246)	112.0
  (22, 2076)	138.0
  (635, 17906)	7.0
  (94, 9299)	339.0
  (94, 9969)	339.0
  (313, 3190)	0.01
  (813, 1005)	4.0
  (250, 36)	57.0
  (250, 3873)	57.0
  (216, 74)	58.0
  (183, 3600)	441.0
  (183, 2109)	441.0
  (1467, 17224)	46.0
  (364, 7502)	505.0
  (364, 13885)	505.0
  (364, 9299)	505.0
  (364, 1289)	505.0
  (106, 17907)	2.0
